In [4]:
%load_ext autoreload
%autoreload 2

#SFTP 

# # Use
# cnopts = pysftp.CnOpts()
# cnopts.hostkeys = None


# #Get Data from PS Data Export Manager SFTP folder, and move to BigQuery
# with pysftp.Connection(
#     host="ps.com",
#     username="greendot",
#     password="*********",
#     cnopts=cnopts
# ) as sftp:
#     # Download a remote file to the local machine
#     remote_file = "/greendottn/custom_report_standards_2024"
#     local_file = "local_file.csv"   (This can be dynamic if we want to preserve the file everytime)
#     sftp.get(remote_file, local_file)

# ----------------------------------------------------------

from modules.buckets import create_bucket, upload_to_bucket, download_from_bucket, upload_to_bq_table
import os
import pandas_gbq
import pandas as pd


# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable in order to interact
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'powerschool-420113-b0750ef59b13.json'



class Create:

    def __init__(self, bucket, end_file_name, local_file_name, project_id, db, table_name, location=None):
        
        self.location = location
        self.bucket = bucket
        self.end_file_name = end_file_name
        self.local_file_name = local_file_name
        self.project_id = project_id
        self.db = db
        self.table_name = table_name


    def process(self):
    
        #Create the bucket, and upload to that bucket. If already created, bypass
        # create_bucket(self.bucket, self.location)

        #Upload file to bucket, demonstrates if overwritten or newfile. 
        #End File Name,  Local File Path, Bucket Name
        # upload_to_bucket(self.end_file_name , self.local_file_name, self.bucket)


        upload_to_bq_table(cloud_storage_uri = f'gs://{self.bucket}/{self.end_file_name}',
                        project_id = self.project_id,
                        db = self.db,
                        table_name = self.table_name,
                        location = self.location)
        
        SQL_query = f'''
            SELECT * FROM {self.project_id}.{self.db}.{self.table_name}
        '''

        try:
            print('Running query')
            query = pandas_gbq.read_gbq(SQL_query, project_id=self.project_id, location=self.location)
            print('Query completed')
        except:
            print('Unable to run query')


        return(query)


# SELECT * FROM `greendotdataflow.Students.Student_Records_10` LIMIT 1000
# SELECT * FROM `powerschool-420113.students_1.students` LIMIT 1000

# project_id = 'powerschool-420113'
# db = 'students_1'
# table_name = 'students'





already_created_update = Create(location = 'us-south1',
                                bucket='psholdingbucket7',
                                end_file_name = 'students.csv',
                                local_file_name = 'Student_Records.csv',
                                project_id='powerschool-420113',
                                db = 'students_1',
                                table_name = 'students'
                    )

# brand_new_creation = Create(location = 'us-south1',
#                             bucket='psholdingbucket12',
#                             end_file_name = 'students.csv',
#                             local_file_name = 'Student_Records.csv',
#                             project_id='greendotdataflow',
#                             db = 'Students',
#                             table_name = 'Student_Records_12',

#                             SQL_query='''

#                             SELECT * FROM greendotdataflow.Students.Student_Records_12

#                             '''  
#                     )


query_one = already_created_update.process()
# query_two = brand_new_creation.process()

#-------------Next step put file in clevers SFTP--------------

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


OSError: Forbidden: b/psholdingbucket7/o
iota-data-flow@powerschool-420113.iam.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist).